In [2]:
import pandas as pd
import mysql.connector
import numpy as np
import os

In [4]:
df_Ap_bus = pd.read_csv('df_Ap_bus.csv')
df_As_bus = pd.read_csv('df_As_bus.csv')
df_H_bus = pd.read_csv('df_H_bus.csv')
df_K_bus = pd.read_csv('df_K_bus.csv')
df_Ka_bus = pd.read_csv('df_Ka_bus.csv')
df_R_bus = pd.read_csv('df_R_bus.csv')
df_S_bus = pd.read_csv('df_S_bus.csv')
df_T_bus = pd.read_csv('df_T_bus.csv')
df_U_bus = pd.read_csv('df_U_bus.csv')
df_Wb_bus = pd.read_csv('df_Wb_bus.csv')
df = pd.concat([df_Ap_bus,df_As_bus,df_H_bus,df_K_bus,df_Ka_bus,df_R_bus,df_S_bus,df_T_bus,df_U_bus,df_Wb_bus])
df
Cleaned_df = df.copy()





In [5]:
Cleaned_df['Ratings'] = pd.to_numeric(df['Ratings'], errors='coerce')

# Optionally, replace NaN with a specific value (e.g., 0) or drop them
Cleaned_df['Ratings'].fillna(Cleaned_df['Ratings'].min(), inplace=True)
Cleaned_df['Ratings'].astype(float)

0     4.2
1     3.8
2     3.1
3     3.8
4     3.5
     ... 
80    4.6
81    4.5
82    4.4
83    4.4
84    4.1
Name: Ratings, Length: 2294, dtype: float64

In [6]:
# Drop NaN values from all columns except 'Route_Links'
Cleaned_df.dropna(subset=[col for col in Cleaned_df.columns if col != 'Seats_Available'], inplace=True)

Cleaned_df['Seats_Available']= Cleaned_df['Seats_Available'].fillna('Not Available')

In [7]:
Cleaned_df['Price']= Cleaned_df['Price'].str.replace('INR', '').str.strip()
Cleaned_df['Price']= Cleaned_df['Price'].astype(float,2)

In [8]:
# Establish the MySQL connection (do not specify the database here)
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="Tango@0044.",
    database="REDBUS"
)

# Create a cursor object
My_cursor = conn.cursor()

# Define the query to create the database
create_database_query = "CREATE DATABASE IF NOT EXISTS REDBUS"

# Execute the query
try:
    My_cursor.execute(create_database_query)
    print("Database created successfully or already exists!")
except mysql.connector.Error as err:
    print(f"Error: {err}")

    

Database created successfully or already exists!


In [10]:
# Create the table if it does not exist
My_cursor.execute('''CREATE TABLE IF NOT EXISTS BUS_DETAILS(
                    ID INT AUTO_INCREMENT PRIMARY KEY,
                    Bus_Name VARCHAR(255) NOT NULL,
                    Bus_Type VARCHAR(255) NOT NULL,
                    Departure_Time VARCHAR(255) NOT NULL,
                    Arrival_Time VARCHAR(255) NOT NULL,
                    Travel_Duration VARCHAR(255) NOT NULL,
                    Ratings FLOAT NULL,
                    Price FLOAT NULL,
                    Seats_Available VARCHAR(255) NOT NULL,
                    Route_Name VARCHAR(255) NOT NULL,
                    Route_Links VARCHAR(255) NOT NULL
                    )''')
print('Done!!')

Done!!


In [11]:
#Query
insert_query = '''INSERT INTO BUS_DETAILS(
                Bus_Name,
                Bus_Type,
                Departure_Time,
                Arrival_Time,
                Travel_Duration,
                Ratings,
                Price,
                Seats_Available,
                Route_Name,
                Route_Links)
                VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
data = Cleaned_df.values.tolist()

My_cursor.executemany(insert_query,data)
conn.commit()
print('Done!!')

Done!!
